<a href="https://colab.research.google.com/github/BOD-27/Find_Missing_People/blob/main/Final_Find_Missing_People_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/BOD-27/Find_Missing_People.git
import os
os.chdir('Find_Missing_People')
!unzip /content/Find_Missing_People/FinalTest_dataset.zip
!unzip /content/Find_Missing_People/Final_dataset.zip
!pip install streamlit
!pip install mtcnn
!pip install keras-facenet

Cloning into 'Find_Missing_People'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 41 (delta 16), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (41/41), 30.64 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Archive:  /content/Find_Missing_People/FinalTest_dataset.zip
   creating: FinalTest_dataset/Abdo/
  inflating: FinalTest_dataset/Abdo/Abdo.jpg  
  inflating: FinalTest_dataset/Abdo/Abdo1.png  
  inflating: FinalTest_dataset/Abdo/Abdo2.png  
  inflating: FinalTest_dataset/Abdo/Abdo3.png  
  inflating: FinalTest_dataset/Abdo/Abdo4.png  
  inflating: FinalTest_dataset/Abdo/Abdo5.png  
  inflating: FinalTest_dataset/Abdo/Abdo6.png  
   creating: FinalTest_dataset/Ahmed/
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (1).jpg  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (1).png  
  inflating: FinalTest_dataset/Ahmed/AhmedBIG (2).png  
  inflating: FinalTest_d

In [3]:
!pip install -r /content/drive/MyDrive/LifeSpan_Downloads/Lifespan_Age_Transformation_Synthesis/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 13.7 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=5ab8d8c4071ce41d793d60e01c21eb92deadbb67413cf1b132dc087ca295e009
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [4]:
%%writefile app.py

import os
import sys
import shutil
import streamlit as st
from PIL import Image
import subprocess
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
import numpy as np
import time

from Life_Span_Model import Navigate, import_libraries, set_options, load_data_visualizer, switch_model_gender, process_image_and_save_result
from Face_Recognition import import_libraries as fr_import_libraries, suppress_tf_warnings, create_embedding_save_directory, FaceLoading, find_closest_matches, check_for_match

def main():
    image = Image.open("/content/Find_Missing_People/Image.png")

    # Resize the image to a smaller size
    image = image.resize((600, 600))

    # Create a container for the title and image
    with st.container():
        # Display the title and image side-by-side
        cols = st.columns([4, 1])
        with cols[0]:
            st.title("Re-Union Families")
        with cols[1]:
            st.image(image, use_column_width=True)

    # File upload
    uploaded_file = st.file_uploader("Upload your image", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        # Save the uploaded file to disk
        img_path = os.path.join("/content", uploaded_file.name)
        with open(img_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Display the uploaded image
        if uploaded_file is not None:
            image = Image.open(uploaded_file)

            # Resize the image to a specific size
            image = image.resize((400, 400))

            st.image(image, caption="Uploaded Image", width=400)

        # Gender selection
        selected_gender = st.selectbox("Select gender:", ["Male", "Female"])

        # Button to run the models
        if st.button("Search"):
            with st.spinner('In Progress...'):
                Navigate()
                import_libraries()
                opt = set_options()
                dataset, visualizer = load_data_visualizer(opt)
                model = switch_model_gender(selected_gender, opt)
                image_folder_path = process_image_and_save_result(img_path, dataset, model, visualizer)
            success_placeholder = st.empty()
            success_placeholder.success('Image processed and saved!')
            time.sleep(5)

            success_placeholder.empty()
            with st.spinner('In Progress...'):
                fr_import_libraries()
                suppress_tf_warnings()
                embedding_save_directory = create_embedding_save_directory()

                face_loader = FaceLoading('/content/Find_Missing_People/Final_dataset')
                embedded_x, dataset_labels = face_loader.load_or_encode_and_save_embeddings(embedding_save_directory)

                face_loader2 = FaceLoading(image_folder_path)
                X1, y1 = face_loader2.LoadClasses()

                # Debugging information
                print(f"Loaded images: {len(X1)}")

                if len(X1) == 0:
                    st.error("No images found in the processed image folder. Please check the uploaded image and try again.")
                    return

                embedder = FaceNet()
                embedded_x1 = [face_loader2.GetEmbedding(embedder, img) for img in X1]

                top_matches = find_closest_matches(embedded_x1, embedded_x, dataset_labels, top_n=7)

                st.sidebar.write("Top Matches:")
                for i, (label, distance) in enumerate(top_matches, 1):
                    st.sidebar.write(f"Similar Image {i}: Label - {label}, Similarity Score - {distance}")
                match_found, avg_similarity, matched_label = check_for_match(top_matches)

            if match_found:
                st.success(f"We successfully found your beloved {matched_label}")
                st.write(f"Average cosine similarity: {avg_similarity}")
                st.success(f"Congratulations!")
                st.balloons()
            else:
                st.error("We are sorry no match found, But We will find him soon.")
                st.write("Adding embeddings...")
                for i, img in enumerate(X1):
                    embedding = face_loader2.GetEmbedding(embedder, img)
                    label = y1[i]
                    np.save(os.path.join(embedding_save_directory, f'{label}embeddings.npy'), embedding)

if __name__ == "__main__":
    main()

Writing app.py


In [5]:
!wget -q -O - ipv4.icanhazip.com


34.16.194.81


In [6]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.194.81:8501

npx: installed 22 in 7.067s
your url is: https://chatty-cars-smash.loca.lt
2024-06-04 21:02:00.398431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 21:02:00.398488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 21:02:00.400560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 21:02:00.414044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimize